### Use OData to query our datastore for matching data

You can use standard OData queries to filter and sort the data in the datastore.  The following example shows a couple different ways to query the datastore using OData.

In [ ]:
from typing import List
from ni.datastore import Client
from ni.datastore.data import PublishedMeasurement
from ni.measurements.data.v1.data_store_pb2 import Outcome

client = Client()

def print_data(data: List[PublishedMeasurement]):
    print("Matching data count: ", len(data))
    for item in data:
        print("  Name: ", item.measurement_name)
        print("  Id: ", item.published_measurement_id)
        print("  Timestamp: ", item.start_date_time)
        print("  Passed: ", item.outcome == Outcome.OUTCOME_PASSED)
        print()

data = client.query_measurements(odata_query="$filter=Name eq 'data publish sample'")
print_data(list(data))

### Use OData to query our custom metadata

You can use OData to query into our custom metadata.  The following shows how to query the custom metadata in a way that is consistent with other OData queries.

In [ ]:
# here's the odata query: look for data with associated hardware that has a cable length of 1.5
query = "$filter=operator/extensions/badge_number eq 'emp-128256'"
data = client.query_measurements(odata_query=query)
print_data(list(data))